To enable autocomplete in kaggel just run this in consol

%config Completer.use_jedi = False

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

Read and see the first five rows of the train dataset.

In [ ]:
df= pd.read_csv('/kaggle/input/health-insurance-cross-sell-prediction/train.csv')
df.head()

check shape of the dataset.

In [ ]:
df.shape
df.info()

# For EDA I have used Dtale but soon update the notebook with EDA and visulizations



let's undrstand the data.

Id :column is unique identifier hence it has no role in prediction

Gender: Male 206089, female 175020

Age: understand the distribution

Driving_license: represent weather the owner posses the lincence or not. My undestanding of the domain says for insurance driving license is required. only 812 observations do not posses  driving license else do.

Region_code: Although looks like integer but it is a categorical data consisting of 53 unique values.

Vehicle age: categorical data 1-2 Year 200316, < 1 Year 164786,> 2 Years 16007 

vehicle damage: categorical data Yes 192413, No 188696










In [ ]:
df.Gender.value_counts()

In [ ]:
sns.displot(df.Age)

In [ ]:
df.Driving_License.value_counts()

In [ ]:
df.Region_Code.nunique()

In [ ]:
df.Previously_Insured.value_counts()

In [ ]:
df.Vehicle_Age.value_counts()

In [ ]:
df.Vehicle_Damage.value_counts()

In [ ]:
df.Annual_Premium.describe()

In [ ]:
df.Policy_Sales_Channel.nunique()

In [ ]:
df.Response.value_counts()


# It is an imbalanced problem.Will use oversampling/undersampling for balancing the data.
> Data Preprocessing.


Drop id, policy sales channel and vintage,Region code

One hot encode gender,vehical damage,driving license,previously insured.

ordinal encode vehical_age.

bin and encode Annual premium and Age.

response is a target.

In [ ]:
df1= df.copy()

In [ ]:
def data_prep(df):
    df= df.drop(columns=['id','Policy_Sales_Channel','Vintage'])
    df=pd.get_dummies(df,columns=['Gender'] ,prefix='Gender')
    df=pd.get_dummies(df,columns=['Vehicle_Damage'] ,prefix='Damage')
    df=pd.get_dummies(df,columns=['Driving_License'] ,prefix='License')
    df=pd.get_dummies(df,columns=['Previously_Insured'] ,prefix='prev_insured')
    df["Age"] = pd.cut(df['Age'], bins=[0, 29, 35, 50, 100])
    df['Age']= df['Age'].cat.codes
    df['Annual_Premium'] = pd.cut(df['Annual_Premium'], bins=[0, 30000, 35000,40000, 45000, 50000, np.inf])
    df['Annual_Premium']= df['Annual_Premium'].cat.codes
    df['Vehicle_Age'] =df['Vehicle_Age'].map({'< 1 Year': 0, '1-2 Year': 1, '> 2 Years': 2})
    df.drop(columns=['Region_Code'],inplace= True)
    return df

In [ ]:
df1=data_prep(df1)
df1.head()

# Features to be used in modelling

In [ ]:
Features= ['Age','Vehicle_Age','Annual_Premium',"Gender_Female","Gender_Male","Damage_No","Damage_Yes","License_0","License_1" ,"prev_insured_0", "prev_insured_1"]


# use train_test split when not using pycaret

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df1[Features],df1['Response'], test_size = 0.3, random_state = 101)
X_train.shape,X_test.shape


# Handle Imbalance using imblearn undersampling

In [ ]:
!pip install imblearn

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
RUS = RandomUnderSampler(sampling_strategy=.5,random_state=3,)
X_train,Y_train  = RUS.fit_resample(df1[Features],df1['Response'])
X_train.head()


In [ ]:
Y_train.value_counts()



# Accuracy metrices

In [ ]:
def performance_met(model,X_train,Y_train,X_test,Y_test):
    acc_train=accuracy_score(Y_train, model.predict(X_train))
    f1_train=f1_score(Y_train, model.predict(X_train))
    acc_test=accuracy_score(Y_test, model.predict(X_test))
    f1_test=f1_score(Y_test, model.predict(X_test))
    print("train score: accuracy:{} f1:{}".format(acc_train,f1_train))
    print("test score: accuracy:{} f1:{}".format(acc_test,f1_test))
    

In [ ]:
from sklearn.metrics import accuracy_score, f1_score,auc
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train,Y_train) 
performance_met(model,X_train,Y_train,X_test,Y_test)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model_DT=DecisionTreeClassifier(random_state=1)
model_DT.fit(X_train,Y_train)
performance_met(model_DT,X_train,Y_train,X_test,Y_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
Forest= RandomForestClassifier(random_state=1)
Forest.fit(X_train,Y_train)
performance_met(Forest,X_train,Y_train,X_test,Y_test)


In [ ]:
from sklearn.model_selection import GridSearchCV
rf= RandomForestClassifier(random_state=1)
parameters = {
    'bootstrap': [True],
'max_depth': [10, 20],
'min_samples_leaf': [3, 4],
'min_samples_split': [4, 6],
'n_estimators': [100, 200],

}
grid_search_1 = GridSearchCV(rf, parameters, cv=3, verbose=2, n_jobs=-1)
grid_search_1.fit(X_train, Y_train)
performance_met(grid_search_1,X_train,Y_train,X_test,Y_test)

In [ ]:
X_train['Response']=Y_train


# Let's use pycaret

In [ ]:
pip install pycaret


In [ ]:
from pycaret.classification import *
clf1= setup(data=X_train,target='Response',data_split_stratify=True)


In [ ]:
compare_models(exclude=['xgboost'])

# Gradient Boosting classifier is the best performer, let's further work with it.

In [ ]:
gbm= create_model('gbc')

In [ ]:
predict_model(gbm,probability_threshold=.65)


In [ ]:
plot_model(gbm)

In [ ]:
plot_model(gbm,'confusion_matrix')

In [ ]:
plot_model(gbm,'feature')

Instead of plotting everything. Just evaluate the model.

In [ ]:
evaluate_model(gbm)

# Now let's have the test data and do the preprocessing to make it a fit for our model.

In [ ]:
test_df= pd.read_csv('../input/health-insurance-cross-sell-prediction/test.csv')
test_df.info()

In [ ]:
test_df2=data_prep(test_df)

In [ ]:
prediction=predict_model(gbm,data=test_df2)

In [ ]:
submission=pd.read_csv('../input/health-insurance-cross-sell-prediction/sample_submission.csv')
submission['Response']=prediction['Label']


In [ ]:
submission.Response.value_counts()

In [ ]:
submission.to_csv('final_submission.csv')

In [ ]:
submission.head(10)

# work in progress...